In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from gensim.parsing.preprocessing import strip_non_alphanum, strip_punctuation
import torch
import torchtext
from sklearn.model_selection import *
from torch import nn
import gensim.downloader as api

from torchtext.data import Field, LabelField
from torchtext.data import BucketIterator
from torchtext.datasets import IMDB
from transformers import BertTokenizer
from transformers import BertModel
from skorch import NeuralNetClassifier
from skorch.callbacks import Freezer
from sklearn.ensemble import *
from sklearn.metrics import *
from sklearn.pipeline import * 
from sklearn.compose import *
from sklearn.linear_model import *
from sklearn.naive_bayes import *
from sklearn.neural_network import *
from sklearn.preprocessing import *
from skorch.callbacks import ProgressBar
from util import *
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import fse
from fse.models import uSIF

from tqdm.auto import tqdm
from tqdm.utils import CallbackIOWrapper
from skopt.space import Real, Integer
from skopt import BayesSearchCV

In [2]:
from gensim.models import KeyedVectors


In [3]:
wv = KeyedVectors.load("/var/patentmark/older/patent-vectors.v4.gensim")

In [ ]:
wv.index2word[1000]

In [4]:
usif = uSIF(wv, workers=32, lang_freq="en")

In [5]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")

In [6]:
subset = list((set(tier1)-set(["PersonalizedProduct"]))&set(all_tiers_100))
subset

['Manufacturing',
 'SpecificationofUse',
 'Imaging',
 'AnalysisAndModeling',
 'SurgicalMethod',
 'AnatomicalTarget']

In [7]:
training_labels = training_set[subset]
testing_labels = testing_set[subset]

In [11]:
tfidf_default_settings = {
    'lowercase': True, 
    'strip_accents': 'ascii',
    #'stop_words' : stopwords,
    'min_df': 5,
    'max_df': 0.5,
    #'ngram_range': (1,3)
}

transformer = ColumnTransformer([
#      ('top_terms', CountVectorizer(analyzer=iden, binary=True, min_df=2), 'top_terms'),
#      ('cited_by', CountVectorizer(analyzer=iden, binary=True, min_df=2), 'cited_by'),
#      ('inventors', CountVectorizer(analyzer=iden, binary=True, min_df=2), 'inventors'),
#      ('citations', CountVectorizer(analyzer=citations_split, binary=True, min_df=2), 'citations'),
#      ('similar_patents', CountVectorizer(analyzer=iden, binary=True, min_df=2), 'similar_patents'),
#      ('cpc', CountVectorizer(analyzer=cpc_split, binary=True, min_df=2), 'cpc_codes'),
#      ('embedding_v1', Extract(), 'embedding_v1'),
     ('usif_abstract', Embedder(usif), 'abstract'),
     ('usif_claims', Embedder(usif), 'claims'),
     ('usif_description', Embedder(usif), 'description'),
#      ('abstract_tfidf', TfidfVectorizer(**tfidf_default_settings), 'abstract'),
#      ('claims_tfidf', TfidfVectorizer(**tfidf_default_settings), 'claims'),
#      ('description_tfidf', TfidfVectorizer(**tfidf_default_settings), 'description')
    ], verbose=False, n_jobs=-1)



transformer_grid = {
#     'top_terms__min_df': (1, 5),
#     'top_terms__max_df': Real(0.1, 1.0),
#     'cited_by__min_df': (1, 5),
#     'cited_by__max_df': Real(0.1, 1.0),
#     'inventors__min_df': (1, 20),
#     'inventors__max_df': Real(0.1, 1.0),
#     'citations__min_df': (1, 20),
#     'citations__max_df': Real(0.1, 1.0),
#     'similar_patents__min_df': (1, 20),
#     'similar_patents__max_df': Real(0.1, 1.0),
#     'cpc__min_df': (1, 20),
#     'cpc__max_df': Real(0.1, 1.0),
#     'abstract_tfidf__min_df': (1, 20),
#     'abstract_tfidf__max_df': Real(0.1, 1.0),
#     'claims_tfidf__min_df': (1, 20),
#     'claims_tfidf__max_df': Real(0.1, 1.0),
#     'description_tfidf__min_df': (1, 20),
#     'description_tfidf__max_df': Real(0.1, 1.0),
}



In [12]:
pipe = Pipeline(steps=[('transformer', transformer),
                       #('svd', TruncatedSVD(random_state=42, n_components=1024)),
                       #('dummy', OneVsRestClassifier(DummyClassifier()))
                       #('svc', OneVsRestClassifier(SVC(random_state=42), n_jobs=-1))
                       ('rf',  RandomForestClassifier(n_jobs=-1, random_state=42))
                       #('lr', OneVsRestClassifier(LogisticRegression(n_jobs=-1), n_jobs=-1))
                       #('cat', OneVsRestClassifier(CatBoostClassifier(verbose=True)))
                       #('knn', KNeighborsClassifier(n_jobs=-1))
                      ], 
                verbose=True,
                memory="cachedir/")

model_grid = {
    'svd__n_components': (8, 1024),
    'rf__max_depth': (1, 500), #[10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 175, 200, 250, 300, 350, 400, None],
    'rf__min_samples_leaf': (1, 12),
    'rf__min_samples_split': (2, 12),
    'rf__n_estimators': (5, 10000)          
}

In [13]:
pipe.fit(training_set, training_labels)
predictions = pipe.predict(testing_set)
print(classification_report(testing_labels, predictions, target_names=subset))

[Pipeline] ....... (step 1 of 2) Processing transformer, total=  37.8s


/home/martin/anaconda3/envs/phenetics/lib/python3.8/site-packages/sklearn/pipeline.py:309: UserWarning: Persisting input arguments took 91.09s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  X, fitted_transformer = fit_transform_one_cached(


[Pipeline] ................ (step 2 of 2) Processing rf, total=   0.3s
                     precision    recall  f1-score   support

      Manufacturing       0.39      0.27      0.31        83
 SpecificationofUse       0.42      0.20      0.27        79
            Imaging       0.59      0.65      0.62       133
AnalysisAndModeling       0.47      0.20      0.28        84
     SurgicalMethod       0.86      0.15      0.26        40
   AnatomicalTarget       0.71      0.83      0.76       164

          micro avg       0.59      0.49      0.53       583
          macro avg       0.57      0.38      0.42       583
       weighted avg       0.57      0.49      0.50       583
        samples avg       0.60      0.50      0.51       583



/home/martin/anaconda3/envs/phenetics/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/martin/anaconda3/envs/phenetics/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
param_grid = {**model_grid, **({f"transformer__{k}": v for k,v in transformer_grid.items()})}
param_grid

In [ ]:
search = BayesSearchCV(pipe, param_grid, n_iter=50, n_points=3, pre_dispatch=36, refit=True, cv=3, verbose=10, random_state=42, n_jobs=-1)

with tqdm(total=search.total_iterations) as pbar:
    def on_step(optim_result):
        print(optim_result)
        pbar.update(9)
        return False
    search.fit(training_set, training_labels, callback=on_step)

In [ ]:
print(search.best_params_)

In [ ]:
predictions =  search.best_estimator_.predict(testing_set)
print(classification_report(testing_labels, predictions, target_names=subset))

In [ ]:
OrderedDict([('rf__max_depth', 10), ('rf__min_samples_leaf', 1), ('rf__min_samples_split', 12), ('rf__n_estimators', 1000), ('svd__n_components', 64)])

                     precision    recall  f1-score   support

            Imaging       0.59      0.90      0.71       133
     SurgicalMethod       0.86      0.15      0.26        40
AnalysisAndModeling       0.00      0.00      0.00        84
 SpecificationofUse       0.64      0.09      0.16        79
   AnatomicalTarget       0.69      0.96      0.80       164
      Manufacturing       0.48      0.13      0.21        83

          micro avg       0.63      0.52      0.57       583
          macro avg       0.54      0.37      0.35       583
       weighted avg       0.54      0.52      0.46       583
        samples avg       0.64      0.55      0.55       583

In [ ]:
# Untuned RF with uSIF (Glove)

#      SurgicalMethod       1.00      0.03      0.05        40
#    AnatomicalTarget       0.68      0.95      0.79       164
#             Imaging       0.55      0.78      0.64       133
#  SpecificationofUse       0.50      0.04      0.07        79
# AnalysisAndModeling       0.20      0.01      0.02        84
#       Manufacturing       0.57      0.10      0.16        83

#           micro avg       0.61      0.47      0.53       583
#           macro avg       0.58      0.32      0.29       583
#        weighted avg       0.56      0.47      0.41       583
#         samples avg       0.63      0.50      0.52       583